# Volatility Modeling Using ARCH, GARCH & EWMA
This notebook demonstrates how to model financial market volatility using **ARCH**, **GARCH**, and **EWMA** models.

In [1]:
import numpy as np
import pandas as pd
import requests
from io import StringIO

import yfinance as yf
from arch import arch_model
from arch import arch_model

## Download data

In [2]:
api_key = 'SLK1N6T6LSBSR8NK'  # (Replace with your own key if needed)
symbol = 'JPM'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&datatype=csv'
response = requests.get(url)

df = pd.read_csv(StringIO(response.text))
df.index = df['timestamp']
df = df.loc[:'2015-01-01']
df = df['close']
df

timestamp
2025-10-13    307.97
2025-10-10    300.89
2025-10-09    305.53
2025-10-08    304.03
2025-10-07    307.69
               ...  
2025-05-28    263.49
2025-05-27    265.29
2025-05-23    260.71
2025-05-22    260.67
2025-05-21    261.04
Name: close, Length: 100, dtype: float64

## ARCH Model

In [3]:
df = yf.download('JPM', start='2022-01-01', end='2025-01-01')
df['returns'] = df['Close'].pct_change() * 100  # daily % return
returns = df['returns'].dropna() #drop missing values

model = arch_model(returns, vol='ARCH', p=1)
results = model.fit(disp='off') #Create and fit ARCH(1) model

results.summary() #Show summary

C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\3600702999.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('JPM', start='2022-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


<class 'statsmodels.iolib.summary.Summary'>
"""
                      Constant Mean - ARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -1400.95
Distribution:                  Normal   AIC:                           2807.90
Method:            Maximum Likelihood   BIC:                           2821.76
                                        No. Observations:                  752
Date:                Tue, Oct 14 2025   Df Residuals:                      751
Time:                        19:56:18   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0978  5.518e-02      1.772  7.646e-02 [-1.039e-02,  0.206]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          2.1877      0.332      6.584  4.591e-11    [  1.536,  2.839]
alpha[1]       0.1224  7.467e-02      1.639      0.101 [-2.394e-02,  0.269]
===========================================================================

Covariance estimator: robust
"""

In [4]:
forecast = results.forecast(horizon=5) #Forecast 5 days ahead
predicted_variance = forecast.variance
predicted_volatility = predicted_variance ** 0.5
#predicted_volatility


predicted_volatility = [1.479269, 1.567023, 1.577429, 1.578698, 1.578853] # Calculate average of predicted volatility
predicted_avg_vol = sum(predicted_volatility) / len(predicted_volatility)
#predicted_avg_vol

start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')  # buffer for weekends and holidays
real_df = yf.download('JPM', start_date, end_date)
real_df['returns'] = real_df['Close'].pct_change() * 100 # Calculate realized volatility
real_df = real_df.dropna()

realized_vol = real_df['returns'].std() * np.sqrt(5)

print("ARCH Model Predicted Volatility:", predicted_avg_vol)
print("ARCH Model Actual Volatility:", realized_vol)

C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\1083604519.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed

ARCH Model Predicted Volatility: 1.5562543999999998
ARCH Model Actual Volatility: 1.6919531359238498


## GARCH Model

In [5]:
df = yf.download('JPM', start='2022-01-01', end='2025-01-01')

df['returns'] = df['Close'].pct_change() * 100 #Calculate daily returns

returns = df['returns'].dropna() #Drop missing values

model = arch_model(returns, vol='GARCH', p=1, q=1) # Create and fit GARCH(1,1) model
results = model.fit(disp='off')

results.summary()

C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\2699400612.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('JPM', start='2022-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1387.47
Distribution:                  Normal   AIC:                           2782.94
Method:            Maximum Likelihood   BIC:                           2801.43
                                        No. Observations:                  752
Date:                Tue, Oct 14 2025   Df Residuals:                      751
Time:                        19:56:18   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1146  5.754e-02      1.991  4.648e-02 [1.786e-03,  0.227]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0220  1.702e-02      1.294      0.196 [-1.134e-02,5.537e-02]
alpha[1]       0.0165  1.112e-02      1.481      0.139 [-5.326e-03,3.827e-02]
beta[1]        0.9737  1.217e-02     79.976      0.000      [  0.950,  0.998]
=============================================================================

Covariance estimator: robust
"""

In [6]:
forecast = results.forecast(horizon=5) #Forecast 5 days ahead
predicted_variance = forecast.variance
predicted_volatility = predicted_variance ** 0.5
predicted_volatility

predicted_volatility = [1.623483, 1.622252, 1.621032, 1.619824, 1.618626] #Calculate average of predicted volatility
predicted_avg_vol = sum(predicted_volatility) / len(predicted_volatility)
predicted_avg_vol

start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-09')

real_df = yf.download('JPM', start_date, end_date) #Realized volatility
real_df['returns'] = real_df['Close'].pct_change() * 100
real_df = real_df.dropna()

realized_vol = real_df['returns'].std() * np.sqrt(5)

print("GARCH Model Predicted Volatility:", predicted_avg_vol)
print("GARCH Model Actual Volatility:", realized_vol)

C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\216661278.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('JPM', start_date, end_date) #Realized volatility
[*********************100%***********************]  1 of 1 completed

GARCH Model Predicted Volatility: 1.6210434
GARCH Model Actual Volatility: 1.6919531359238498


## EWMA Model

In [7]:
df = yf.download('JPM', start='2022-01-01', end='2025-01-01')

df['returns'] = df['Close'].pct_change() #Daily returns
df = df.dropna()

lamda = 0.94 #Set lambda for EWMA

ewma_var = []
var_t = df['returns'].var() #Initialize variance

for ret in df['returns']:
    variance_tplus1 = lamda * var_t + (1 - lamda) * (ret ** 2)
    ewma_var.append(variance_tplus1) #Determine EWMA

df['ewma_vol'] = np.sqrt(ewma_var)

latest_daily_vol = df['ewma_vol'].iloc[-1] #Predicted volatility
latest_daily_vol

start_date = pd.to_datetime('2024-12-31')
end_date = pd.to_datetime('2025-01-03')

real_df = yf.download('JPM', start_date, end_date)
real_df['returns'] = real_df['Close'].pct_change() * 100
real_df = real_df.dropna() #Real volatility

realized_vol = real_df['returns']
#realized_vol


C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\2137982070.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('JPM', start='2022-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed
C:\Users\Startklar\AppData\Local\Temp\ipykernel_6680\2137982070.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('JPM', start_date, end_date)
[*********************100%***********************]  1 of 1 completed
